# Test of HiPace300 Pumps on Transfer and Depo Chamber

## Equipment Configuration:
- **HiPace300 Pumps**: Connected to Transfer and Deposition Chambers
- **HiScroll12**: One unit connected as backing pump
- **Pressure Sensors**: Two total sensors connected to both Omnicontrol 300 units

## Test Overview:
This notebook contains tests and monitoring for the HiPace300 turbo molecular pump system with integrated pressure monitoring and control capabilities.

## 1. Import Required Libraries and Setup


In [1]:
import sys
import threading
import time
from datetime import datetime
import asyncio
from typing import Dict, Optional, Any
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import ipywidgets as widgets
import pickle

from loguru import logger
import os
from pathlib import Path

# Add path to src modules
sys.path.append(os.path.join(os.getcwd(), '..', '..', 'src'))

# Import current device modules
from devices.pfeiffer.hiscroll12.hiscroll12 import HiScroll12
from devices.pfeiffer.hipacebus import HiPace300Bus

## 2. Setup Shared Logging System

In [2]:
# Get the repository root and create shared logs directory
repo_root = Path(os.getcwd()).parent.parent
log_dir = repo_root / "debugging" / "logs"
log_dir.mkdir(parents=True, exist_ok=True)

# Create shared log file for all devices
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
shared_log_file = log_dir / f"hipace_Transfer_Depo_test_{timestamp}.log"

# Configure shared logger
logger.remove()  # Remove default logger

# Add console logger with INFO level
logger.add(sys.stderr, level="INFO", format="{time:YYYY-MM-DD HH:mm:ss} | {level} | {message}")

# Add shared file logger with DEBUG level
logger.add(
    str(shared_log_file),
    level="DEBUG",
    format="{time:YYYY-MM-DD HH:mm:ss.SSS} | {level} | {name}:{function}:{line} | {message}",
    rotation="1 day",
    retention="30 days",
    compression="zip"
)

logger.info("Pump Locker Test system initialized with shared logging")
print(f"Repository root: {repo_root}")
print(f"Shared logs will be saved to: {shared_log_file}")

2025-08-13 09:26:56 | INFO | Pump Locker Test system initialized with shared logging


Repository root: C:\Users\ESIBDlab\PycharmProjects\esibd_bs
Shared logs will be saved to: C:\Users\ESIBDlab\PycharmProjects\esibd_bs\debugging\logs\hipace_Transfer_Depo_test_20250813_092656.log


# 3. Set COM-Ports & Adresses

In [3]:
# Device configuration dictionaries
# Modify these COM ports and addresses according to your actual hardware setup

HiScroll12_dict = {
    'com_port': 'COM19',
    'device_address': 2,
    'device_type': 'HiScroll12',
    'description': 'Backing pump for HiPace300 systems'
}

HiPace300_Transfer_dict = {
    'com_port': 'COM36',
    'device_address': 101,
    'omnicontrol_address': 101,
    'tc400_address':1,
    'gauge1_address': 122,
    'device_type': 'HiPace300',
    'description': 'Transfer chamber turbo pump'
}

HiPace300_Depo_dict = {
    'com_port': 'COM37',
    'device_address': 101,
    'omnicontrol_address': 101,
    'tc400_address':1,
    'gauge1_address': 122,
    'device_type': 'HiPace300', 
    'description': 'Deposition chamber turbo pump'
}

# Print configuration summary
print("Device Configuration Summary:")
print("=" * 40)
print(f"HiScroll12 (Backing pump):")
print(f"  COM Port: {HiScroll12_dict['com_port']}")
print(f"  Address: {HiScroll12_dict['device_address']}")
print(f"  Description: {HiScroll12_dict['description']}")

print(f"\nHiPace300 Transfer:")
print(f"  COM Port: {HiPace300_Transfer_dict['com_port']}")
print(f"  Address: {HiPace300_Transfer_dict['device_address']}")
print(f"  Description: {HiPace300_Transfer_dict['description']}")

print(f"\nHiPace300 Depo:")
print(f"  COM Port: {HiPace300_Depo_dict['com_port']}")
print(f"  Address: {HiPace300_Depo_dict['device_address']}")
print(f"  Description: {HiPace300_Depo_dict['description']}")

print("\n✅ Device dictionaries configured successfully")

Device Configuration Summary:
HiScroll12 (Backing pump):
  COM Port: COM19
  Address: 2
  Description: Backing pump for HiPace300 systems

HiPace300 Transfer:
  COM Port: COM36
  Address: 101
  Description: Transfer chamber turbo pump

HiPace300 Depo:
  COM Port: COM37
  Address: 101
  Description: Deposition chamber turbo pump

✅ Device dictionaries configured successfully


# 4. Connect Devices

In [ ]:
# Initialize and connect HiScroll12
hiscroll = HiScroll12(
    device_id="hiscroll12",
    port=HiScroll12_dict['com_port'],
    device_address=HiScroll12_dict['device_address'],
    logger=logger
)
hiscroll.connect()
print(f"✅ HiScroll12 connected on {HiScroll12_dict['com_port']}")

In [10]:
# Initialize and connect HiPace300 Transfer
hipace_transfer = HiPace300Bus(
    device_id="hipace_transfer",
    port=HiPace300_Transfer_dict['com_port'],
    device_address=HiPace300_Transfer_dict['device_address'],
    omnicontrol_address=HiPace300_Transfer_dict['omnicontrol_address'],
    tc400_address=HiPace300_Transfer_dict['tc400_address'],
    gauge1_address=HiPace300_Transfer_dict['gauge1_address'],
    logger=logger
)

hipace_transfer.connect()
print(f"✅ HiPace300 Transfer connected on {HiPace300_Transfer_dict['com_port']}")

2025-08-13 09:31:36 | INFO | Connecting to Pfeiffer device hipace_transfer on COM36
2025-08-13 09:31:36 | INFO | Successfully connected to device at address 101


✅ HiPace300 Transfer connected on COM36


In [23]:
# Initialize and connect HiPace300 Depo
hipace_depo = HiPace300Bus(
    device_id="hipace_depo",
    port=HiPace300_Depo_dict['com_port'],
    device_address=HiPace300_Depo_dict['device_address'],
    omnicontrol_address=HiPace300_Depo_dict['omnicontrol_address'],
    tc400_address=HiPace300_Depo_dict['tc400_address'],
    gauge1_address=HiPace300_Depo_dict['gauge1_address'],
    logger=logger
)

hipace_depo.connect()
print(f"✅ HiPace300 Depo connected on {HiPace300_Depo_dict['com_port']}")

2025-08-13 10:10:55 | INFO | Connecting to Pfeiffer device hipace_depo on COM37
2025-08-13 10:10:55 | INFO | Successfully connected to device at address 101


✅ HiPace300 Depo connected on COM37


# 5. Debugging/Testing Connection manually

In [26]:
hiscroll.disconnect()
hipace_depo.disconnect()
hipace_transfer.disconnect()

2025-08-13 10:23:21 | INFO | Disconnected from Pfeiffer device hiscroll12
2025-08-13 10:23:21 | INFO | Disconnected from Pfeiffer device hipace_depo
2025-08-13 10:23:21 | INFO | Disconnected from Pfeiffer device hipace_transfer


True